# Lecture 13

## Repetition: Object oriented programming
A good guide: https://realpython.com/python3-object-oriented-programming/

We start by creating a class. A class is a recipe to create objects of a specific type.

For example, we want to have a database of students:

In [29]:
MONTHLY_STUDENT_LOAN = 8000


class Student:
    """
    Parameters
    ----------
    name : str
        Name of student
    age : int
        Age of student
    transcript : dict[str, str]
        Dictionary mapping class names to grades
    current_classes : list[str]
        List of currently enrolled classes
    
    
    Attributes
    ----------
    name : str
        Name of student
    age : int
        Age of student
    transcript : dict[str, str]
        Dictionary mapping class names to grades
    current_classes : set[str]
        List of currently enrolled classes
    money : int
        How much money the student has
    has_graduated : bool
    salary : int (default=0)
        The monthly salary of the student
    help_from_parents : int (default=0)
        The amount of money the student gets from their parents each month.
    """
    def __init__(self, name, age, transcript=None, current_classes=None, salary=0, help_from_parents=0):
        if transcript is None:
            transcript = {}
        if current_classes is None:
            current_classes = set()
        self.name = name
        self.age = age
        self.transcript = transcript
        self.current_classes = current_classes
        self.has_graduated = False
        self.money = 0
        self.salary = salary
        self.help_from_parents = help_from_parents
    
    def add_class(self, new_class):
        """Add new class to the current classes.
        """
        if new_class in current_classes:
            raise ValueError(f'{self.name} is already taking that class.')
        self.current_classes.add(new_class)
    
    def get_student_loan(self):
        self.money += MONTHLY_STUDENT_LOAN
    
    def get_salary(self):
        self.money += self.salary
    
    def get_help_from_parents(self):
        self.money += self.help_from_parents

    def get_monthly_income(self):
        self.get_student_loan()
        self.get_salary()
        self.get_help_from_parents()
        
    def check_is_working(self):
        return self.salary > 0

In [30]:
nils = Student(name='Nils Olsen', age=21)
print(nils.money)
nils.get_monthly_income()
print(nils.money)

0
8000


Next, we create the database

In [31]:
class DataBase:
    """
    Parameters
    ----------
    student_params : list[dict]
        Dictionary with keyword arguments to pass to Student generation
    """
    def __init__(self, student_params):
        self.students = {}
        self.next_idx = 0
        for student_kwargs in student_params:
            self.add_student(Student(**student_kwargs))
    
    def add_student(self, student):
        self.students[self.next_idx] = student
        self.next_idx += 1
    
    def __repr__(self):
        return repr(self.students)
    
    def __str__(self):
        return f'Student database: \n {str(self.students)}'

In [32]:
students = [
    {
        'name': 'Nils',
        'age': 21
    },
    {
        'name': 'Ananya',
        'age': 23,
        'transcript': {'MATH111': 'A', 'PHI100': 'C', 'IMRT100': 'Pass', 'STAT100': 'A'}
    }
]

student_database = DataBase(students)

In [33]:
print(student_database)

Student database: 
 {0: <__main__.Student object at 0x7f90154900f0>, 1: <__main__.Student object at 0x7f9015490fd0>}


But now, we wish to extend the code above to have lecturers as well!

In [44]:
MONTHLY_STUDENT_LOAN = 8000


class Lecturer:
    """
    Parameters
    ----------
    name : str
        Name of student
    age : int
        Age of student
    transcript : dict[str, str]
        Dictionary mapping class names to grades
    current_classes : list[str]
        List of currently enrolled classes
    
    
    Attributes
    ----------
    name : str
        Name of student
    age : int
        Age of student
    money : int (default=0)
        How much money the lecturer has
    salary : int (default=0)
        The monthly salary of the lecturer
    loans : int (default=0)
        The monthly payment on loans
    """
    def __init__(self, name, age, money=0, salary=0, loans=0, classes=None):
        self.name = name
        self.age = age
        self.has_graduated = True
        self.money = money
        self.salary = salary
        self.loans = loans
        
        if classes is None:
            classes = []
        self.classes = classes
    
    def get_salary(self):
        self.money += self.salary
    
    def pay_loans(self):
        self.money -= self.loans

    def get_monthly_income(self):
        self.get_salary()
        self.pay_loans()
        
    def check_is_working(self):
        return True

Here, we see that there is much repeated code! We should have spent time thinking about the program before we started coding!

Let us create a parent class, person, that contain the shared code!

In [69]:
class Person:
    """
    Parameters
    ----------
    name : str
        Name of student
    age : int
        Age of student
    transcript : dict[str, str]
        Dictionary mapping class names to grades
    current_classes : list[str]
        List of currently enrolled classes
    
    
    Attributes
    ----------
    name : str
        Name of student
    age : int
        Age of student
    money : int
        How much money the lecturer has
    has_graduated : bool
    salary : int (default=0)
        The monthly salary of the lecturer
    loans : int (default=0)
        The monthly payment on loans
    """
    def __init__(self, name, age, salary=0, loans=0):
        self.name = name
        self.age = age
        self.money = 0
        self.salary = salary
        self.loans = loans
    
    def get_salary(self):
        self.money += self.salary
    
    def pay_loans(self):
        self.money -= self.loans

    def get_monthly_income(self):
        self.get_salary()
        self.pay_loans()
    
    def check_is_working(self):
        return self.salary > 0
    
    def check_has_loans(self):
        return self.loans > 0


class Student(Person):
    """
    Parameters
    ----------
    name : str
        Name of student
    age : int
        Age of student
    transcript : dict[str, str]
        Dictionary mapping class names to grades
    current_classes : list[str]
        List of currently enrolled classes
    
    
    Attributes
    ----------
    name : str
        Name of student
    age : int
        Age of student
    transcript : dict[str, str]
        Dictionary mapping class names to grades
    current_classes : set[str]
        List of currently enrolled classes
    money : int
        How much money the student has
    has_graduated : bool
    salary : int (default=0)
        The monthly salary of the student
    help_from_parents : int (default=0)
        The amount of money the student gets from their parents each month.
    """
    def __init__(self, name, age, transcript=None, current_classes=None, salary=0, help_from_parents=0):
        super().__init__(
            name=name,
            age=age,
            salary=salary,
            loans=0,
        )
        self.help_from_parents = help_from_parents
        
        if transcript is None:
            transcript = {}
        if current_classes is None:
            current_classes = set()
        self.transcript = transcript
        self.current_classes = current_classes
        self.has_graduated = False
        
    
    def add_class(self, new_class):
        """Add new class to the current classes.
        """
        if new_class in current_classes:
            raise ValueError(f'{self.name} is already taking that class.')
        self.current_classes.add(new_class)
    
    def get_student_loan(self):
        self.money += MONTHLY_STUDENT_LOAN
    
    def get_help_from_parents(self):
        self.money += self.help_from_parents

    def get_monthly_income(self):
        super().get_monthly_income()
        self.get_salary()
        self.get_help_from_parents()

        
class Lecturer(Person):
    def __init__(self, name, age, salary=35_000, loans=0, classes=None,):
        super().__init__(
            name=name,
            age=age,
            salary=salary,
            loans=loans
        )
        if classes is None:
            classes = []
        self.classes = classes

In [70]:
nils = Student(name='Nils', age=21)

In [71]:
yngve = Lecturer(name='Yngve', age=26, salary=29_000, loans=4_000, classes=['INF200'])

In [72]:
yngve.money

0

Her ser vi at koden ble mye kortere! La oss også utvide databasen til å kunne takle forelesere og studenter.

In [73]:
class DataBase:
    """
    Parameters:
    -----------
    student_params : list[tuple[str, dict]]
        List of tuples whose first element is a key in the valid_types dict
        and second element is a keyword-argument dictionary.
    
    Attributes:
    -----------
    valid_types : dict[str, type]
    """
    
    valid_types = {
        'Lecturer': Lecturer,
        'Student': Student,
        'Person': Person
    }  # This can be done more elegantly with a separate register class.
    
    def __init__(self, person_params):
        self.persons = {}
        self.next_idx = 0
        for person in person_params:
            self.add_person(*person)
    
    def add_person(self, person_type, person_kwargs):
        PersonType = self.valid_types[person_type]
        self.persons[self.next_idx] = PersonType(**person_kwargs)
        self.next_idx += 1
    
    def __repr__(self):
        return repr(self.persons)
    
    def __str__(self):
        return f'Person database: \n {str(self.persons)}'

In [74]:
persons = [
    (
        'Student',
         {
            'name': 'Nils',
            'age': 21
        },
    ),
    (
        'Student',
        {
            'name': 'Ananya',
            'age': 23,
            'transcript': {'MATH111': 'A', 'PHI100': 'C', 'IMRT100': 'Pass', 'STAT100': 'A'}
        },
    ),
    (
        'Lecturer',
        {
            'name': 'Yngve',
            'age': 23,
            'salary': 29_000,
            'loans': 4_000,
        }
    )
]

database = DataBase(persons)

In [75]:
database

{0: <__main__.Student object at 0x7f901414d780>, 1: <__main__.Student object at 0x7f901414d080>, 2: <__main__.Lecturer object at 0x7f901414db00>}

# One small improvement: Properties
Read about them here: https://www.programiz.com/python-programming/property

In [76]:
yngve = Lecturer('Yngve', 26)
print(yngve.check_has_loans())

False


Here we are using a getter, that is a function that acts as an attribute. Let us create a quick example

In [81]:
import numpy as np

class LineSegment:
    def __init__(self, start, end):
        self.start = np.asarray(start)
        self.end = np.asarray(end)
    
    def interpolate(self, t):
        return (1-t)*self.start + t*self.end
    
    def get_midpoint(self):
        return self.interpolate(0.5)

In [82]:
line_segment = LineSegment([0, 0], [1, 1])

In [83]:
line_segment.get_midpoint()

array([0.5, 0.5])

This is an example of a getter. Let us use properties instead!

In [85]:
import numpy as np

class LineSegment:
    def __init__(self, start, end):
        self.start = np.asarray(start)
        self.end = np.asarray(end)
    
    def interpolate(self, t):
        return (1-t)*self.start + t*self.end
    
    @property
    def midpoint(self):
        return self.interpolate(0.5)

In [86]:
line_segment = LineSegment([0, 0], [1, 1])

In [87]:
line_segment.midpoint  # Notice, no function call. 

array([0.5, 0.5])

We can also have setters, can be useful if we wish to log when something change

In [90]:
class Person:
    def __init__(self, age):
        self._age = age
        self.age_changed = False

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        self._age = value
        self.age_changed = True

In [91]:
person = Person(10)

In [92]:
print(person.age)
print(person.age_changed)
person.age = 15
print(person.age)
print(person.age_changed)

10
False
15
True
